# Training GCN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch import autograd

import dgl
from dgl import DGLGraph
from dgl.data import MiniGCDataset
import dgl.function as fn
from dgl.data.utils import load_graphs

import numpy as np
import pandas as pd

import spacy
import collections

import os

Using backend: pytorch


## Load Data

In [2]:
size = 'xs'

In [3]:
cls_tokens = torch.load('data/X_train_cls_tokens_{}.bin'.format(size))
gcn_offsets = torch.load("data/X_train_gcn_offsets_{}.bin".format(size))
all_graphs, _ = load_graphs("data/X_train_graphs_{}.bin".format(size))

y_data = torch.load('data/y_{}.pt'.format(size))

# Model Building

## RGCN Layer

In [4]:
class RGCNLayer(nn.Module):
    def __init__(self, feat_size, num_rels, activation=None, gated = True):
        
        super(RGCNLayer, self).__init__()
        self.feat_size = feat_size
        self.num_rels = num_rels
        self.activation = activation
        self.gated = gated

        self.weight = nn.Parameter(torch.Tensor(self.num_rels, self.feat_size, 256))
        # init trainable parameters
        nn.init.xavier_uniform_(self.weight,gain=nn.init.calculate_gain('relu'))
        
        if self.gated:
            self.gate_weight = nn.Parameter(torch.Tensor(self.num_rels, self.feat_size, 1))
            nn.init.xavier_uniform_(self.gate_weight,gain=nn.init.calculate_gain('sigmoid'))
        
    def forward(self, g):
        
        weight = self.weight
        gate_weight = self.gate_weight
        
        def message_func(edges):
            w = weight[edges.data['rel_type']]
            msg = torch.bmm(edges.src['h'].unsqueeze(1), w).squeeze()
            msg = msg * edges.data['norm']
            
            if self.gated:
                gate_w = gate_weight[edges.data['rel_type']]
                gate = torch.bmm(edges.src['h'].unsqueeze(1), gate_w).squeeze().reshape(-1,1)
                gate = torch.sigmoid(gate)
                msg = msg * gate
                
            return {'msg': msg}
    
        def apply_func(nodes):
            h = nodes.data['h']
            h = self.activation(h)
            return {'h': h}
        g.update_all(message_func, fn.sum(msg='msg', out='h'), apply_func)

## Define Full RGCN Model

In [5]:
class RGCNModel(nn.Module):
    def __init__(self, h_dim, num_rels, num_hidden_layers=1, gated = True):
        super(RGCNModel, self).__init__()

        self.h_dim = h_dim
        self.num_rels = num_rels
        self.num_hidden_layers = num_hidden_layers
        self.gated = gated
        
        # create rgcn layers
        self.build_model()
       
    def build_model(self):        
        self.layers = nn.ModuleList() 
        for _ in range(self.num_hidden_layers):
            rgcn_layer = RGCNLayer(self.h_dim, self.num_rels, activation=F.relu, gated = self.gated)
            self.layers.append(rgcn_layer)
    
    def forward(self, g):
        for layer in self.layers:
            layer(g) # todo: maybe g = layer(g)??
        
        rst_hidden = []
        for sub_g in dgl.unbatch(g):
            rst_hidden.append(  sub_g.ndata['h']   )
        return rst_hidden

## Design the Main Model (R-GCN + FFNN)

In [6]:
class Head(nn.Module):
    """The MLP submodule"""
    def __init__(self, gcn_out_size: int, bert_out_size: int):
        super().__init__()
        self.bert_out_size = bert_out_size
        self.gcn_out_size = gcn_out_size
        
        self.fc = nn.Sequential(
            nn.BatchNorm1d(bert_out_size + gcn_out_size * 3),
            nn.Dropout(0.5),
            nn.Linear(bert_out_size + gcn_out_size * 3, 256),    
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, 2), # todo: make sure 2 is fine.
        )
        for i, module in enumerate(self.fc):
            if isinstance(module, (nn.BatchNorm1d, nn.BatchNorm2d)):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.Linear):
                if getattr(module, "weight_v", None) is not None:
                    nn.init.uniform_(module.weight_g, 0, 1)
                    nn.init.kaiming_normal_(module.weight_v)
                    assert model[i].weight_g is not None
                else:
                    nn.init.kaiming_normal_(module.weight)
                nn.init.constant_(module.bias, 0)
                
    def forward(self, gcn_outputs, offsets_gcn, bert_embeddings):
        
        gcn_extracted_outputs = [gcn_outputs[i].unsqueeze(0).gather(1, offsets_gcn[i].unsqueeze(0).unsqueeze(2)
                                       .expand(-1, -1, gcn_outputs[i].unsqueeze(0).size(2))).view(gcn_outputs[i].unsqueeze(0).size(0), -1) for i in range(len(gcn_outputs))]
        
        gcn_extracted_outputs = torch.stack(gcn_extracted_outputs, dim=0).squeeze()
        
        embeddings = torch.cat((gcn_extracted_outputs, bert_embeddings), 1) 
        
        return self.fc(embeddings)    
    
    
class GPRModel(nn.Module):
    """The main model."""
    def __init__(self):
        super().__init__()
        self.RGCN =  RGCNModel(h_dim = 1024, num_rels = 3, gated = True)
#         self.BERThead = BERT_Head(1024) # bert output size
        self.head = Head(256, 1024)  # gcn output   berthead output
    
    
    def forward(self, g, offsets_gcn, cls_token):
        gcn_outputs = self.RGCN(g)
#         print(gcn_outputs.shape)
        bert_head_outputs = cls_token
        head_outputs = self.head(gcn_outputs, offsets_gcn, bert_head_outputs)
        return head_outputs

In [ ]:
class Head2(nn.Module):
    """The MLP submodule"""
    def __init__(self, gcn_out_size: int, bert_out_size: int):
        super().__init__()
        self.bert_out_size = bert_out_size
        self.gcn_out_size = gcn_out_size
        
        self.fc = nn.Sequential(
            nn.BatchNorm1d(bert_out_size),
            nn.Dropout(0.5),
            nn.Linear(bert_out_size, 256),    
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, 2), # todo: make sure 2 is fine.
        )
        for i, module in enumerate(self.fc):
            if isinstance(module, (nn.BatchNorm1d, nn.BatchNorm2d)):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.Linear):
                if getattr(module, "weight_v", None) is not None:
                    nn.init.uniform_(module.weight_g, 0, 1)
                    nn.init.kaiming_normal_(module.weight_v)
                    assert model[i].weight_g is not None
                else:
                    nn.init.kaiming_normal_(module.weight)
                nn.init.constant_(module.bias, 0)
                
    def forward(self, bert_embeddings):
        
#         gcn_extracted_outputs = [gcn_outputs[i].unsqueeze(0).gather(1, offsets_gcn[i].unsqueeze(0).unsqueeze(2)
#                                        .expand(-1, -1, gcn_outputs[i].unsqueeze(0).size(2))).view(gcn_outputs[i].unsqueeze(0).size(0), -1) for i in range(len(gcn_outputs))]
        
#         gcn_extracted_outputs = torch.stack(gcn_extracted_outputs, dim=0).squeeze()
        
#         embeddings = torch.cat((gcn_extracted_outputs, bert_embeddings), 1) 
        
        return self.fc(bert_embeddings)    
    
    
class GPRModel2(nn.Module):
    """The main model."""
    def __init__(self):
        super().__init__()
        self.RGCN =  RGCNModel(h_dim = 1024, num_rels = 3, gated = True)
#         self.BERThead = BERT_Head(1024) # bert output size
        self.head = Head2(256, 1024)  # gcn output   berthead output
    
    
    def forward(self, cls_token):
#         gcn_outputs = self.RGCN(g)
#         print(gcn_outputs.shape)
        bert_head_outputs = cls_token
        head_outputs = self.head(bert_head_outputs)
        return head_outputs

In [7]:
class GPRDataset(Dataset):
    def __init__(self, graphs, gcn_offsets, cls_tokens, labels):

        self.graphs = graphs
        self.cls_tokens = cls_tokens
        self.gcn_offsets = gcn_offsets
        self.y = labels
        
    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], self.gcn_offsets[idx], self.cls_tokens[idx], self.y[idx]

In [8]:
train_dataset = GPRDataset(all_graphs, gcn_offsets, cls_tokens, y_data)

In [9]:
def collate(samples):
    
    graphs, gcn_offsets, cls_tokens, labels = map(list, zip(*samples))
    
    batched_graph = dgl.batch(graphs)
    offsets_gcn = torch.stack([torch.LongTensor(x) for x in gcn_offsets], dim=0)
    
    cls_tokens = torch.stack(cls_tokens, dim=0).squeeze()
    
    labels = torch.stack(labels)
    
    return batched_graph, offsets_gcn, cls_tokens, labels

In [18]:
train_dataloader = DataLoader(
   train_dataset,
   collate_fn = collate,
   batch_size = 64,
   shuffle=True,
)

In [19]:
model = GPRModel()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-5)
# optimizer = optim.SGD(model.parameters(), lr=1e-5, momentum=0.9)

reg_lambda = 0.035

save_model_name = "models/gcn_model_{}.pt".format(size)

for epoch in range(100):  # loop over the dataset multiple times

        running_loss = 0.0
        with autograd.detect_anomaly():
            for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
                graphs, gcn_offsets, cls_tokens, labels = data
    #             inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
        
                outputs = model(graphs, gcn_offsets, cls_tokens)

#                 l2_reg = None
#                 for w in model.RGCN.parameters():
#                     if not l2_reg:
#                         l2_reg = w.norm(2)
#                     else:
#                         l2_reg = l2_reg + w.norm(2)  
#                 for w in model.head.parameters():
#                     if not l2_reg:
#                         l2_reg = w.norm(2)
#                     else:
#                         l2_reg = l2_reg + w.norm(2)

#                 loss = criterion(outputs, labels) + l2_reg * reg_lambda
                loss = criterion(outputs, labels)
                loss.backward()
                
                optimizer.step()
                # print statistics

                running_loss += loss.item()

                if i % 4 == 3:    # print every 5 mini-batches
                    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss/4))
                    running_loss = 0.0

        torch.save(model, save_model_name)

[1,     4] loss: 1.378
[2,     4] loss: 1.054
[3,     4] loss: 0.979
[4,     4] loss: 0.772
[5,     4] loss: 0.837
[6,     4] loss: 0.811
[7,     4] loss: 0.788
[8,     4] loss: 0.742
[9,     4] loss: 0.733
[10,     4] loss: 0.745
[11,     4] loss: 0.728
[12,     4] loss: 0.706
[13,     4] loss: 0.697
[14,     4] loss: 0.725
[15,     4] loss: 0.742
[16,     4] loss: 0.659
[17,     4] loss: 0.678
[18,     4] loss: 0.696
[19,     4] loss: 0.669
[20,     4] loss: 0.737
[21,     4] loss: 0.685
[22,     4] loss: 0.696
[23,     4] loss: 0.669
[24,     4] loss: 0.637
[25,     4] loss: 0.656
[26,     4] loss: 0.634
[27,     4] loss: 0.665
[28,     4] loss: 0.662
[29,     4] loss: 0.683
[30,     4] loss: 0.621
[31,     4] loss: 0.641
[32,     4] loss: 0.627
[33,     4] loss: 0.603
[34,     4] loss: 0.619
[35,     4] loss: 0.635
[36,     4] loss: 0.618
[37,     4] loss: 0.592
[38,     4] loss: 0.617
[39,     4] loss: 0.654
[40,     4] loss: 0.643
[41,     4] loss: 0.629
[42,     4] loss: 0.620
[